# Analysis of NYC public and charter schools results in ELA and math grades 6-8.

## Generating the map

#### Data sources:

**1) State test Math and ELA results (2022-2023)**

Report Card Database (251.35 megabytes): This Access database contains assessment results (elementary- and intermediate-level ELA, Math, and Science; Annual Regents; Total Cohort Regents; NYSESLAT; NYSAA), for the state, districts, public with charter schools, by county, and Need to Resource Capacity group.
https://data.nysed.gov/downloads.php

**2) Schools locations**

NYS GIS Clearinghouse: NYS Schools
https://data.gis.ny.gov/maps/b6c624c740e4476689aa60fdc4aacb8f/about

#### Definitions of Performance Levels for the 2023 Grades 3-8 English Language Arts and Mathematics Tests  

**NYS Level 1**: Students performing at this level are below proficient in standards for their grade. They may demonstrate limited knowledge, skills, and practices embodied by the Learning Standards that are considered insufficient for the expectations at this grade. 

**NYS Level 2**: Students performing at this level are partially proficient in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered partial but insufficient for the expectations at this grade. Students performing at Level 2 are considered on track to meet current New York high school graduation requirements but are not yet proficient in Learning Standards at this grade. 

**NYS Level 3**: Students performing at this level are proficient in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered sufficient for the expectations at this grade.  

**NYS Level 4**: Students performing at this level excel in standards for their grade. They demonstrate knowledge, skills, and practices embodied by the Learning Standards that are considered more than sufficient for the expectations at this grade.  

*Source: NYSED, 2023, https://www.p12.nysed.gov/irs/ela-math/2023/ela-math-score-ranges-performance-levels-2023.pdf*

### Imports

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from shapely.geometry import Point
from fuzzywuzzy import process
import fuzzywuzzy
import base64
from io import BytesIO
import math
from tqdm import tqdm
import importlib
import utils
importlib.reload(utils)
from utils import create_plot, match_name

pd.set_option('display.float_format', '{:.3f}'.format)

### Read data

In [ ]:
basePath = r"G:\My Drive\Kids\NYC_schools_mapped"
dataFolder = r"raw_data"
outputFolder = r"processed_data"

In [ ]:
# Read GeoJSON into data frame
SchoolsFile = 'NYS_Schools.geojson'
NYCSchoolsPath = os.path.join(basePath, dataFolder, SchoolsFile)
NYCSchoolsGeom = gpd.read_file(NYCSchoolsPath)

# DistrictsFile = 'School Districts.geojson'
# NYCDistrictsPath = os.path.join(basePath, dataFolder, DistrictsFile)
# NYCDistrictsGeom = gpd.read_file(NYCDistrictsPath)

In [ ]:
## Read schools test results files

# read schools math results file
fileName_math = "NYS_MS_MATH_from_NYS.xlsx"
mathPath = os.path.join(basePath,dataFolder,fileName_math)
print(mathPath)
mathResultsDF = pd.read_excel(mathPath)

# read schools ELA results file
fileName_ELA = "NYS_MS_ELA_from_NYS.xlsx"
ELAPath = os.path.join(basePath, dataFolder, fileName_ELA)
print(ELAPath)
ELAResultsDF = pd.read_excel(ELAPath)

In [ ]:
# ## Read district results files

# # Read file with district wide Math test results to add to the map
# DistrictMathFile = "DistrictsMSMAthNorm.xlsx"
# DistrictMathPath = os.path.join(basePath, outputFolder, DistrictMathFile)
# DistrictMSMathData = pd.read_excel(DistrictMathPath)
# print(DistrictMSMathData.head(5))

# # Read file with district wide ELA test results to add to the map
# DistrictELAFile = "DistrictsMSELANorm.xlsx"
# DistrictELAPath = os.path.join(basePath, outputFolder, DistrictELAFile)
# DistrictMSELAData = pd.read_excel(DistrictELAPath)
# print(DistrictMSELAData.head(5))

In [ ]:
mathResultsDF.info()

In [ ]:
ELAResultsDF.info()

In [ ]:
mathResultsDF.rename(columns = {'YEAR':'Year'}, inplace = True)
mathResultsDF.info()

### Prepare school layer

In [ ]:
# Get locations for public schools only 
#(select only public schools (public, charter, charter, SATELLITE SITE FOR CHARTER SCHOOLS) from geoJSON)

NYCSchoolsGeom = NYCSchoolsGeom[NYCSchoolsGeom['INST_TYPE_DESC'] == 'PUBLIC SCHOOLS']
NYCSchoolsGeom

In [ ]:
# Make a dataframe from geoJSON with minimum needed columns

NYCSchoolsGeom_short = NYCSchoolsGeom[['OBJECTID', 'LEGAL_NAME', 'INSTSUBTYPDESC', 'SDL_DESC', 'geometry']]
NYCSchoolsGeom_short

In [ ]:
name = 'NYSPubChSchools_temp.csv'
path = os.path.join(basePath, outputFolder, name)
NYCSchoolsGeom_short.to_csv(path)

del name, path

In [ ]:
# Dictionnary for schools test results results
subjects = ['Math', 'ELA']
resultsDFs = {'Math': mathResultsDF, 'ELA': ELAResultsDF}

In [ ]:
# resultsDF.info() showed that most of the columns are objects instead of numbers and needed to be converted
for subject in subjects:
    resultsDF = resultsDFs[subject]
    resultsDF_colToConvert = ['LEVEL1_COUNT',
     'LEVEL2_COUNT',                             
     'LEVEL3_COUNT',
     'LEVEL4_COUNT',
     ]
    resultsDF[resultsDF_colToConvert] = resultsDF[resultsDF_colToConvert].apply(pd.to_numeric, errors = 'coerce')
    resultsDF.info()
    print(len(resultsDF))
    
del resultsDF

In [ ]:
for subject in subjects:
    resultsDF = resultsDFs[subject]
    resultsDF = resultsDF[['ENTITY_NAME', 'Year', 'ASSESSMENT_NAME', 'LEVEL1_COUNT', 'LEVEL2_COUNT', 'LEVEL3_COUNT', 'LEVEL4_COUNT']]
    resultsDF.info()
    resultsDFs[subject] = resultsDF
    print(len(resultsDF))
    
del resultsDF

In [ ]:
results_AVG2y = {}

for subject in subjects:
        
    resultsDF = resultsDFs[subject]
    
    resultsDF_grouped = resultsDF.groupby(['ENTITY_NAME'])[['LEVEL1_COUNT', 'LEVEL2_COUNT', 'LEVEL3_COUNT', 'LEVEL4_COUNT']].sum()
    # Change column names to include subject
    resultsDF_grouped.columns = [f'Level 1 {subject}',f'Level 2 {subject}',f'Level 3 {subject}',f'Level 4 {subject}']
    
    # Dataframe for middle schools by years with normalized values
    results_AVG2y[subject] = resultsDF_grouped.div(resultsDF_grouped.sum(axis=1), axis=0)
    results_AVG2y[subject].reset_index(inplace=True)
    
    print(results_AVG2y[subject].head(20))
    
    # Dataframe with average
    
del resultsDF, resultsDF_grouped

In [ ]:
results_Norm = {}

for subject in subjects:
        
    resultsDF = resultsDFs[subject]
    
    resultsDF_grouped = resultsDF.groupby(['ENTITY_NAME', 'Year'])[['LEVEL1_COUNT', 'LEVEL2_COUNT', 'LEVEL3_COUNT', 'LEVEL4_COUNT']].sum()
    # Change column names to include subject
    resultsDF_grouped.columns = [f'Level 1 {subject}',f'Level 2 {subject}',f'Level 3 {subject}',f'Level 4 {subject}']
    
    # Dataframe for middle schools by years with normalized values
    results_Norm[subject] = resultsDF_grouped.div(resultsDF_grouped.sum(axis=1), axis=0)
    results_Norm[subject].reset_index(inplace=True)
    
    print(results_Norm[subject].head(20))
    
    # Dataframe with average
    
del resultsDF, resultsDF_grouped

In [ ]:
# Make a merged dataframe with both Math and ELA results for 2y average
DFs = list(results_AVG2y.values())
allResultsDFAVG2y = pd.merge(DFs[0], DFs[1], on = ['ENTITY_NAME'], how = 'inner')
allResultsDFAVG2y.head(5)

In [ ]:
allResultsDFAVG2y.info()

In [ ]:
# Make a merged dataframe with both Math and ELA results
DFs = list(results_Norm.values())
allResultsDF = pd.merge(DFs[0], DFs[1], on = ['ENTITY_NAME', 'Year'], how = 'inner')
allResultsDF.head(5)

In [ ]:
allResultsDF.info()

In [ ]:
print(allResultsDF.duplicated(subset='ENTITY_NAME').sum())

In [ ]:
allResultsDFAVG2y['Level 4 Math+Ela'] = allResultsDFAVG2y[f'Level 4 {subjects[0]}']+allResultsDFAVG2y[f'Level 4 {subjects[1]}']
allResultsDFAVG2y.head(5)

In [ ]:
allResultsDF['Level 4 Math+Ela'] = allResultsDF[f'Level 4 {subjects[0]}']+allResultsDF[f'Level 4 {subjects[1]}']
allResultsDF.head(10)

In [ ]:
allResultsDF.info()

In [ ]:
name = 'NYSPubChSchoolsTestResults2023_temp.csv'
path = os.path.join(basePath, outputFolder, name)
allResultsDF.to_csv(path)

del name, path

In [ ]:
# Make plots for popups in the map and add them as columns to the mappable dataframe

# Set interactive mode off
plt.ioff()

# list of schools names

schoolsNames = allResultsDF['ENTITY_NAME'].to_list()
testResults = allResultsDF

# Create disctionnary to hold the dataframes by schools
schoolDFs = {}

# Make dataframes by schools 
for name in schoolsNames:
    dfName = name
    schoolDFs[dfName] = testResults[testResults['ENTITY_NAME'] == name]

plots = []
plotsDFs = {}

print("Making test results plots ...")

for subject in subjects:
    columns_to_plot = [f"Level 1 {subject}", f"Level 2 {subject}", f"Level 3 {subject}", f"Level 4 {subject}"]  
    # Plot dataframes by school
    for schoolDF, current_dataframe in tqdm(schoolDFs.items()):
        # schoolDF contains the name of the dataframe
        # current_dataframe contains the dataframe itself

            # Do something with current_dataframe
            # Create a plot
            fig = create_plot(current_dataframe, schoolDF, columns_to_plot)

            # Convert the plot to a PNG image and then encode it
            io_buf = BytesIO()
            fig.savefig(io_buf, format='png', bbox_inches='tight', dpi=85)
            # Close the figure
            plt.close()
            #Reading file to get the base64 string
            io_buf.seek(0)
            base64_string = base64.b64encode(io_buf.read()).decode('utf8')

            pair = (schoolDF, base64_string)

            plots.append(pair)

    # add the plots to the geodataframe of middle schools subject results 
    plotsDFs[subject] = pd.DataFrame(plots, columns=['ENTITY_NAME', f'plot {subject}'])
    
# Concatenate all plots DataFrames along the columns before merging
combined_plots_df = pd.concat(plotsDFs.values(), axis=1)
            
print('Adding plots to the data frame with test results.')    
allResultsDFAVG2y = pd.merge(allResultsDFAVG2y, combined_plots_df, left_on = 'ENTITY_NAME', right_on=combined_plots_df.iloc[:, 0])
print('Done.')    
# Set interactive mode on
# plt.ion()

In [ ]:
allResultsDFAVG2y.info()

In [ ]:
allResultsDF.info()

In [ ]:
allResultsDFAVG2y = allResultsDFAVG2y.drop(['ENTITY_NAME_y', 'ENTITY_NAME_x'], axis = 1)

In [ ]:
allResultsDFAVG2y.head()

In [ ]:
allResultsDF = allResultsDF.drop(['ENTITY_NAME_y', 'ENTITY_NAME_x'], axis=1)

In [ ]:
allResultsDF.head()

In [ ]:
allResultsDF_2023 = allResultsDF[allResultsDF['Year'] == 2023]

In [ ]:
allResultsDF_2023.head()

In [ ]:
# Matching the school all data file average for 2 years 
#with spatial data (geojson of schools locations)

tqdm.pandas(desc="Matching Names")

matched_tuples = allResultsDFAVG2y['ENTITY_NAME'].progress_apply(
    lambda x: match_name(x, NYCSchoolsGeom_short['LEGAL_NAME'], min_score=60))

print('Done.')

In [ ]:
# Matching the school all data file with spatial data (geojson of schools locations)

tqdm.pandas(desc="Matching Names")

matched_tuples = allResultsDF_2023['ENTITY_NAME'].progress_apply(
    lambda x: match_name(x, NYCSchoolsGeom_short['LEGAL_NAME'], min_score=60))

print('Done.')

In [ ]:
allResultsDF_2023.info()

In [ ]:
print('Appending mathes to the dataframe.')
allResultsDFAVG2y['matched_name'] = list(zip(*matched_tuples))[0]
allResultsDFAVG2y['matched_score'] = list(zip(*matched_tuples))[1]
print('Done.')

In [ ]:
print('Appending mathes to the dataframe.')
allResultsDF_2023['matched_name'] = list(zip(*matched_tuples))[0]
allResultsDF_2023['matched_score'] = list(zip(*matched_tuples))[1]
print('Done.')

In [ ]:
# Unmatched or matched incorrectly names identified by 
# visual observations on the map or by analysing the geoJSON in prefered software

unmatched = {
'BGLIG-SHIRLEY RODRIGUEZ-REMENESKI CS':'BRONX GLOBAL LEARNING INSTITUTE FOR GIRLS CHARTER SCHOOL THE SHIRLEY RODRGUEZ-REMENESKI SCHOOL',
'MEADOW HILL GLOBAL EXPLORATIONS MAGN':'',
'SEED HARLEM':'SCHOOL OF EARTH EXPLORATION AND DISCOVERY HARLEM (SEED HARLEM)',
'PS/IS 210 21ST CENTURY ACADEMY':'PS/IS 210 TWENTY-FIRST CENTURY ACADEMY FOR COMMUNITY LEADERSHIP',
'HARBOR HEIGHTS':'HARBOR VIEW SCHOOL (THE)',
'QUEENS COLLEGIATE':'QUEENS COLLEGIATE - A COLLEGE BOARD SCHOOL',
'LAWRENCE ES-BROADWAY':'',
'BROOKLYN EAST COLLEGIATE CS':'',
'COLLEGIATE ACADEMY-MATH-PERSONAL AWA':'COLLEGIATE ACADEMY FOR MATHEMATICS AND PERSONAL AWARENESS CHARTER SCHOOL',
'SOUNDVIEW ACADEMY':'SOUNDVIEW ACADEMY FOR CULTURE AND SCHOLARSHIP',
'MS 224 MANHATTAN EAST':'MS 224 MANHATTAN EAST SCHOOL FOR ARTS & ACADEMICS',
'PATHWAYS COLLEGE PREPARATORY':'PATHWAYS COLLEGE PREPARATORY SCHOOL:  A COLLEGE BOARD SCHOOL',
'30TH AVENUE SCHOOL':'30TH AVENUE SCHOOL (THE) (G & T CITYWIDE)',
'OPPENHEIM-EPHRATAH-ST JOHNSVILLE JS':'OPPENHEIM-EPHRATAH-ST JOHNSVILLE JUNIOR/SENIOR HIGH SCHOOL',
'SCIENCE AND TECHNOLOGY ACADEMY':'SCIENCE AND TECHNOLOGY',
'SULLIVAN WEST HIGH SCHOOL':'SULLIVAN WEST HIGH SCHOOL AT LAKE HUNTINGTON',
}

In [ ]:
# Replacing the erroneus matches in the allResultsDF_2023 data frame

def replace_values(row):
    if row['ENTITY_NAME'] in unmatched:
        row['matched_name'] = unmatched[row['ENTITY_NAME']]
    return row

allResultsDFAVG2y = allResultsDFAVG2y.apply(replace_values, axis = 1)

In [ ]:
# Replacing the erroneus matches in the allResultsDF_2023 data frame

def replace_values(row):
    if row['ENTITY_NAME'] in unmatched:
        row['matched_name'] = unmatched[row['ENTITY_NAME']]
    return row

allResultsDF_2023 = allResultsDF_2023.apply(replace_values, axis = 1)

In [ ]:
name = 'NYSPubChSchoolsTestResults2023_tempMatched.csv'
path = os.path.join(basePath, outputFolder, name)
print(f'Saving to {path} ...')
allResultsDF_2023.to_csv(path)
print('Saved.')
del name, path

# Merging DataFrames based on the matched name

finalGeoDF = pd.merge(NYCSchoolsGeom_short,allResultsDF_2023, left_on='LEGAL_NAME', right_on='matched_name')
allData_Name = 'PublicCharterNYSschools.geojson'
allData_Path = os.path.join(basePath,outputFolder, allData_Name)
print(f'Saving to {allData_Path} ...')
finalGeoDF.to_file(allData_Path, driver="GeoJSON")
print('Saved.')

del allData_Name, allData_Path

In [ ]:
name = 'NYSPubChSchoolsTestResults2yAVG_tempMatched.csv'
path = os.path.join(basePath, outputFolder, name)
print(f'Saving to {path} ...')
allResultsDFAVG2y.to_csv(path)
print('Saved.')
del name, path

# Merging DataFrames based on the matched name

finalGeoDF = pd.merge(NYCSchoolsGeom_short, allResultsDFAVG2y, left_on='LEGAL_NAME', right_on='matched_name')
allData_Name = 'PublicCharterNYSschools2yAVG.geojson'
allData_Path = os.path.join(basePath,outputFolder, allData_Name)
print(f'Saving to {allData_Path} ...')
finalGeoDF.to_file(allData_Path, driver="GeoJSON")
print('Saved.')

del allData_Name, allData_Path

In [ ]:
finalGeoDF.info()

In [ ]:
finalGeoDF.info()

### Generating the map

In [ ]:
from IPython.core.display import display, HTML

display(HTML("<style>.output_scroll { height: auto !important; max-height: 1500px; }</style>"))

# Create a map object, centered at NYC
mapNYS = folium.Map(location=[40.6839, -73.9026], zoom_start=11, tiles="cartodb positron")
   
# Add dataframes with coordinates and test results to the map

def my_style(x):
    level4 = x['properties']['Level 4 Math+Ela']
    charter = x['properties']['INSTSUBTYPDESC']
    color = '#f0a607' if charter == 'CHARTER SCHOOL'  else '#f0a607' if charter == 'SATELLITE SITE FOR CHARTER SCHOOLS' else '#06a6cf'
    #fill_color = '#f0a607' if charter == 'CHARTER SCHOOL'  else '#f0a607' if charter == 'SATELLITE SITE FOR CHARTER SCHOOLS' else '#06a6cf'
    if level4 is None:
        level4 = 0
    #print(level4)
    return {
        "radius": (level4)*500,
        "color": color,
        #"fill_color": fill_color,
    }  


# Function to create iframe for a given row
def create_iframe(row):    
    html =  '<strong>{0}:</strong> {1}<br><strong>{2}:</strong> {3}<br><strong>{4}:</strong> {5}<br>\
    <br><img src="data:image/png;base64,{6}"><br>\
    <img src="data:image/png;base64,{7}">'.format(
        'School Name', row['LEGAL_NAME'],
        'Level 4 share 2022-2023 AVG Math', round(row['Level 4 Math'], 2), 
        'Level 4 share 2022-2023 AVG ELA', round(row['Level 4 ELA'], 2),
        row['plot Math'], row['plot ELA'])
    return folium.IFrame(html, width=500, height=450)

def create_popup(x):
    iframe = create_iframe(x)
    popup = folium.Popup(iframe)
    return popup

# Iterate over the GeoDataFrame and add a popup to each feature
for _, row in tqdm(finalGeoDF.iterrows(), total = len(finalGeoDF)):
    iframe = create_iframe(row)
        
    data = gpd.GeoDataFrame(row.to_frame().T, crs=finalGeoDF.crs)
    
    folium.GeoJson(
    data,
    marker = folium.Circle(radius=10, fill_color='white', fill_opacity=0, color="green", weight=2),
    #marker = folium.Circle(radius=10),    
    popup = folium.Popup(iframe),
    style_function = my_style, 
    control = False    
    #zoom_on_click = True,    
).add_to(mapNYS)    
        
folium.LayerControl().add_to(mapNYS)    
  
# # Display the map
# mapNYC

# Save map to html
mfile = 'NYSpublicAndCharter2yAVG.html'
mpath = os.path.join(basePath, outputFolder, mfile)
print(f'Saving to {mpath} ...')
mapNYS.save(mpath)
print('Saved.')

In [ ]:
from IPython.core.display import display, HTML

display(HTML("<style>.output_scroll { height: auto !important; max-height: 1500px; }</style>"))

# Create a map object, centered at NYC
mapNYS = folium.Map(location=[40.6839, -73.9026], zoom_start=11, tiles="cartodb positron")
   
# Add dataframes with coordinates and test results to the map

def my_style(x):
    level4 = x['properties']['Level 4 Math+Ela']
    charter = x['properties']['INSTSUBTYPDESC']
    color = '#f0a607' if charter == 'CHARTER SCHOOL'  else '#f0a607' if charter == 'SATELLITE SITE FOR CHARTER SCHOOLS' else '#06a6cf'
    #fill_color = '#f0a607' if charter == 'CHARTER SCHOOL'  else '#f0a607' if charter == 'SATELLITE SITE FOR CHARTER SCHOOLS' else '#06a6cf'
    if level4 is None:
        level4 = 0
    #print(level4)
    return {
        "radius": (level4)*500,
        "color": color,
        #"fill_color": fill_color,
    }  


# Function to create iframe for a given row
def create_iframe(row):    
    html =  '<strong>{0}:</strong> {1}<br><strong>{2}:</strong> {3}<br><strong>{4}:</strong> {5}<br>\
    <br><img src="data:image/png;base64,{6}"><br>\
    <img src="data:image/png;base64,{7}">'.format(
        'School Name', row['LEGAL_NAME'],
        'Level 4 share 2023 Math', round(row['Level 4 Math'], 2), 
        'Level 4 share 2023 ELA', round(row['Level 4 ELA'], 2),
        row['plot Math'], row['plot ELA'])
    return folium.IFrame(html, width=500, height=450)

def create_popup(x):
    iframe = create_iframe(x)
    popup = folium.Popup(iframe)
    return popup

# Iterate over the GeoDataFrame and add a popup to each feature
for _, row in tqdm(finalGeoDF.iterrows(), total = len(finalGeoDF)):
    iframe = create_iframe(row)
        
    data = gpd.GeoDataFrame(row.to_frame().T, crs=finalGeoDF.crs)
    
    folium.GeoJson(
    data,
    marker = folium.Circle(radius=10, fill_color='white', fill_opacity=0, color="green", weight=2),
    #marker = folium.Circle(radius=10),    
    popup = folium.Popup(iframe),
    style_function = my_style, 
    control = False    
    #zoom_on_click = True,    
).add_to(mapNYS)    
        
folium.LayerControl().add_to(mapNYS)    
  
# # Display the map
# mapNYC

# Save map to html
mfile = 'NYSpublicAndCharter.html'
mpath = os.path.join(basePath, outputFolder, mfile)
print(f'Saving to {mpath} ...')
mapNYS.save(mpath)
print('Saved.')

In [ ]:
finalGeoDF['SDL_DESC'].unique()

In [ ]:
NYCSchoolsGeom_short['SDL_DESC'].unique()

In [ ]:
NYCSchoolsGeom_short.info()

In [ ]:
import folium
m = folium.Map(location=[40.6839, -73.9026], zoom_start=11, tiles="cartodb positron")

# mfile = 'PublicCharterNYSschools.geojson'
# mpath = os.path.join(basePath, outputFolder, mfile)
# mpath = os.path.join(basePath, dataFolder, SchoolsFile)
# Add GeoJSON as an external file
folium.GeoJson(NYCSchoolsGeom, 
               marker = folium.Circle(radius=10, fill_color='white', fill_opacity=0, color="green", weight=2),
               ).add_to(m)

# Save the map
m.save('map.html')

# del mfile, mpath
# del mpath

In [ ]:
%pwd

In [ ]:
import folium
m = folium.Map(location=[40.6839, -73.9026], zoom_start=11, tiles="cartodb positron")

folium.GeoJson(finalGeoDF, 
               marker = folium.Circle(radius=10, fill_color='white', fill_opacity=0, color="green", weight=2),
               ).add_to(m)

# Save the map
m.save('map2.html')

In [ ]:
finalGeoDF.head()

In [ ]:
import folium
m = folium.Map(location=[40.6839, -73.9026], zoom_start=11, tiles="cartodb positron")

mfile = 'PublicCharterNYSschools.geojson'
mpath = os.path.join(basePath, outputFolder, mfile)
# mpath = os.path.join(basePath, dataFolder, SchoolsFile)
# Add GeoJSON as an external file
folium.GeoJson(mpath, 
               marker = folium.Circle(radius=10, fill_color='white', fill_opacity=0, color="green", weight=2),
               ).add_to(m)

# Save the map
mfile = 'map3.html'
mpath = os.path.join(basePath, outputFolder, mfile)
m.save(mpath)

del mfile, mpath
# del mpath

In [ ]:
import folium
m = folium.Map(location=[40.6839, -73.9026], zoom_start=11, tiles="cartodb positron")

mfile = 'NYS_Schools.geojson'
mpath = os.path.join(basePath, dataFolder, SchoolsFile)

# Add GeoJSON as an external file
folium.GeoJson(mpath,  name='geojson').add_to(m)

# Save the map
m.save('map.html')

del mfile, mpath
# del mpath